In [1]:
from StackGP import *
import sympy as sym

In [2]:
def ComputeSymbolicHess(model,vars):
    printedModel=sym.simplify(printGPModel(model))
    if type(printedModel)==float:
        return sym.matrices.dense.MutableDenseMatrix(np.zeros((vars,vars)))
    hess=sym.hessian(printedModel, [symbols('x'+str(i)) for i in range(vars)])
    return hess

def EvaluateHess(hess,vars,values):
    numHess=hess.subs({symbols('x'+str(j)):values[j] for j in range(vars)})
    hessN = np.array(numHess).astype(float)
    rankN=np.linalg.matrix_rank(hessN)
    return rankN

def Approx2Deriv(model,values,diff1,diff2,positions): #maybe diff should be relative to the variation of each feature
    term1=[values[i]+diff1 if i == positions[0] else values[i] for i in range(len(values))]
    term1=[term1[i]+diff2 if i == positions[1] else term1[i] for i in range(len(term1))]
    term2=[values[i]-diff1 if i == positions[0] else values[i] for i in range(len(values))]
    term2=[term2[i]+diff2 if i == positions[1] else term2[i] for i in range(len(term2))]
    term3=[values[i]+diff1 if i == positions[0] else values[i] for i in range(len(values))]
    term3=[term3[i]-diff2 if i == positions[1] else term3[i] for i in range(len(term3))]
    term4=[values[i]-diff1 if i == positions[0] else values[i] for i in range(len(values))]
    term4=[term4[i]-diff2 if i == positions[1] else term4[i] for i in range(len(term4))]
    return ((evaluateGPModel(model,term1)-evaluateGPModel(model,term2))/((2*diff1))
            -(evaluateGPModel(model,term3)-evaluateGPModel(model,term4))/((2*diff1)))/(2*diff2)


In [109]:
model=initializeGPModels(3,maxLength=20)[0]
printGPModel(model)

exp(-x0 + x2)

In [110]:
hessS=ComputeSymbolicHess(model,3)
hessS

Matrix([
[ exp(-x0 + x2), 0, -exp(-x0 + x2)],
[             0, 0,              0],
[-exp(-x0 + x2), 0,  exp(-x0 + x2)]])

In [117]:
d=0.0001
b=.0001
((evaluateGPModel(model,[1+d+b,2,3])-evaluateGPModel(model,[1-d+b,2,3]))/((2*d))-(evaluateGPModel(model,[1+d-b,2,3])-evaluateGPModel(model,[1-d-b,2,3]))/((2*d)))/(2*b)

7.389056100315372

In [147]:
appx=[[Approx2Deriv(model,[1,2,3],0.0001,0.0001,[i,j]) for i in range(3)] for j in range(3)]
appx

[[7.389056100315372, 0.0, -7.3890562113376745],
 [0.0, 0.0, 0.0],
 [-7.3890562113376745, 0.0, 7.389056122519833]]

In [164]:
rankN=np.linalg.matrix_rank(appx,tol=0.0001*0.0001*10)
rankN

1

In [135]:
EvaluateHess(hessS,3,[1,2,3])

1

In [111]:
numHess=hessS.subs({symbols('x'+str(j)):[1,2,3][j] for j in range(3)})
np.array(numHess).astype(float)

array([[ 7.3890561,  0.       , -7.3890561],
       [ 0.       ,  0.       ,  0.       ],
       [-7.3890561,  0.       ,  7.3890561]])

In [102]:
EvaluateHess(hessS,3,[1,2,3])

KeyboardInterrupt: 

In [43]:
# Convert all values in hessN to float
hessN = np.array(hessN).astype(float)

In [44]:
# Compute the rank of hessN
rankN=np.linalg.matrix_rank(hessN)
rankN

1

In [40]:
[float(hessN[i,j]) for i in range(3) for j in range(3)]

[-12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [32]:
model=initializeGPModels(3,maxLength=20)[0]
printGPModel(model)

(x0 - x1*x2 + 107.876413632784*(1 - 0.17065211943761*x1)**2 - 28.8945435317298)/x1

In [38]:
appx=[[Approx2Deriv(model,[1,2,3],0.0001,0.0001,[i,j]) for i in range(3)] for j in range(3)]
appx

[[0.0, -0.00010089706847793423, 0.0],
 [-0.00010089706847793423, 0.00016768808563938364, 0.0],
 [0.0, 0.0, 1.4210854715202004e-06]]

In [34]:
hessS=ComputeSymbolicHess(model,3)
hessS

Matrix([
[       0,                                                                                                                                                              -1/x1**2, 0],
[-1/x1**2, 6.28318530717959/x1 - 2*(6.28318530717959*x1 - x2 - 36.8186772475258)/x1**2 + 2*(x0 - x1*x2 + 107.876413632784*(0.17065211943761*x1 - 1)**2 - 28.8945435317298)/x1**3, 0],
[       0,                                                                                                                                                                     0, 0]])

In [39]:
numHess=hessS.subs({symbols('x'+str(j)):[1,99,3][j] for j in range(3)})
np.array(numHess).astype(float)

array([[ 0.        , -0.00010203,  0.        ],
       [-0.00010203,  0.00016486,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [42]:
rankN=np.linalg.matrix_rank(appx,tol=0.0001*0.0001*1000)
rankN

2

In [41]:
# Compute the rank of hessN
EvaluateHess(hessS,3,[1,2,3])

2